In [79]:
import pandas as pd
import numpy as np
import json

with open('C:/Users/20234635/OneDrive - TU Eindhoven/Desktop/Code/PCB_Design/RecordFile/PDA_record.json', 'r') as file:
    data = json.load(file)
RECORD_DESIGN = data
SLICE = RECORD_DESIGN

In [80]:
def find_via(trake_o, via_o):
    for i in range(len(trake_o[0])):
        for j in range(len(via_o[0])):
            if via_o[0,j] == None:
                break
            if (trake_o[:2,i].astype(float) == via_o[:2,j]).all():
                trake_o[2,i] = 'Via'
                trake_o[3,i] = via_o[2,j]
    return trake_o

def find_connection(wire, number_track, trake_start_o, trake_end_o, lebal):
    max_lebal = lebal
    for i in range(number_track):
        if i in lebal:
            continue
        else:
            if (wire[:3,-1] == trake_start_o[:3,i]).all():
                wire = np.hstack((wire, trake_end_o[:,i].reshape(-1,1)))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,0] == trake_end_o[:3,i]).all():
                wire = np.hstack((trake_start_o[:,i].reshape(-1,1), wire))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,-1] == trake_end_o[:3,i]).all():
                wire = np.hstack((wire, trake_start_o[:,i].reshape(-1,1)))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,0] == trake_start_o[:3,i]).all():
                wire = np.hstack((trake_end_o[:,i].reshape(-1,1), wire))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            else:
                continue
    return wire, max_lebal

In [81]:
trake_start_o = find_via(trake_start_o, via_o)
trake_end_o = find_via(trake_end_o, via_o)

lebal = []
via_net = []
track_net = []
wire_net = []
number = 0
for i in range(len(SLICE['Record']['Track']['Net'])):
    trake_net_o = SLICE['Record']['Track']['Net'][i]
    trake_start_x_o = SLICE['Record']['Track']['Start X'][i]
    trake_start_y_o = SLICE['Record']['Track']['Start Y'][i]
    trake_end_x_o = SLICE['Record']['Track']['End X'][i]
    trake_end_y_o = SLICE['Record']['Track']['End Y'][i]
    trake_width_o = SLICE['Record']['Track']['Width'][i]
    trake_layer_o = SLICE['Record']['Track']['Layer'][i]
    via_net_o = SLICE['Record']['Via']['Net'][i]
    via_pos_x_o = SLICE['Record']['Via']['Position X'][i]
    via_pos_y_o = SLICE['Record']['Via']['Position Y'][i]
    via_diameter_o = SLICE['Record']['Via']['Diameter'][i]
    trake_start_o = np.vstack([trake_start_x_o, trake_start_y_o, trake_layer_o, trake_width_o, trake_net_o])
    trake_end_o = np.vstack([trake_end_x_o, trake_end_y_o, trake_layer_o, trake_width_o, trake_net_o])
    via_o = np.vstack([via_pos_x_o, via_pos_y_o, via_diameter_o])
    track_o = np.vstack([trake_start_x_o, trake_start_y_o, trake_end_x_o, trake_end_y_o, trake_layer_o, trake_width_o, trake_net_o])

    via_net.append(via_o)
    track_net.append(track_o)

    number_track = len(trake_net_o)
    trake_start_o = find_via(trake_start_o, via_o)
    trake_end_o = find_via(trake_end_o, via_o)

    wires = []
    lebal = []
    number = 0
    for j in range(len(trake_net_o)):
        if j in lebal:
            continue
        else:
            wire = trake_start_o[:,j].reshape(-1,1)
            wire = np.hstack((wire, trake_end_o[:,j].reshape(-1,1)))
            lebal = lebal + [j]
            wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal)
            wires.append(wire.tolist())
            number += 1
    wire_net.append(wires)
print(wire_net[0])

[[[None, None], [None, None], ['None', 'None'], [None, None], ['None', 'None']]]


In [82]:
    def is_column_in_array(checked_matrix, checking_matrix, j):
        column_to_check = checked_matrix[:, j]
    
        for i in range(checking_matrix.shape[1]):
            if np.array_equal(column_to_check, checking_matrix[:, i]):
                return True
        return False
    
    action = []
    step_action = []
    for i in range(len(SLICE['Record']['Track']['Net']) - 1):
        for key, value in SLICE['Record']['Module'].items():
            module_ref = key
            module_o = np.array([value['Position X'][i], value['Position Y'][i], value['Angle'][i]])
            module_p = np.array([value['Position X'][i+1], value['Position Y'][i+1], value['Angle'][i+1]])
            if not np.array_equal(module_o, module_p):
               action.append(['Module', module_ref, (module_p - module_o).tolist()])

        # wire_o = wire_net[i]
        # wire_p = wire_net[i+1]
        # for j in range(len(wire_o)):
        #     if not wire_o[j] in wire_p and wire_o[j][0] != [None, None]:
        #         step_action.append(['Delect Wire', j, wire_o[j]])  
        # for j in range(len(wire_p)):
        #     if not wire_p[j] in wire_o and wire_p[j][0] != [None, None]:
        #         step_action.append(['Add Wire', wire_p[j]])

        track_o = track_net[i]
        track_p = track_net[i+1]
        for j in range(track_o.shape[1]):
            if not is_column_in_array(track_o, track_p, j) and track_o[:,j][0] != None:
                action.append(['Track','Delect', j, track_o[:,j]])  
        for j in range(track_p.shape[1]):
            if not is_column_in_array(track_p, track_o, j) and track_p[:,j][0] != None:
                action.append(['Track', 'Add', track_p[:,j]])
        
        via_o = via_net[i]
        via_p = via_net[i+1]
        for j in range(via_o.shape[1]):
            if not is_column_in_array(via_o, via_p, j) and via_o[:,j][0] != None:
                action.append(['Via', 'Delect', j, via_o[:,j]])  
        for j in range(via_p.shape[1]):
            if not is_column_in_array(via_p, via_o, j) and via_p[:,j][0] != None:
                action.append(['Via', 'Add', via_p[:,j]])

    print(action)

[['Track', 'Add', array(['41.08', '40.03', '52.21', '40.03', 'F.Cu', '0.25', ''],
      dtype='<U32')], ['Track', 'Add', array(['47.91', '44.73', '47.91', '45.1', 'F.Cu', '0.25', ''],
      dtype='<U32')], ['Track', 'Add', array(['52.21', '40.03', '52.41', '40.23', 'F.Cu', '0.25', ''],
      dtype='<U32')], ['Track', 'Add', array(['52.41', '40.23', '48.045', '44.595', 'F.Cu', '0.25', ''],
      dtype='<U32')], ['Track', 'Delect', 1, array(['47.91', '44.73', '47.91', '45.1', 'F.Cu', '0.25', ''],
      dtype='<U32')], ['Via', 'Add', array([44.76, 43.75,  0.8 ])], ['Via', 'Add', array([44.16, 47.29,  0.8 ])], ['Via', 'Delect', 1, array([44.76, 43.75,  0.8 ])]]
